<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/analyse_NY_mobility_poverty_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [ ]:
!pip install ipython-autotime

%load_ext autotime

time: 2.39 ms (started: 2021-04-11 18:47:21 +00:00)


In [ ]:
# Install required dependancies
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 47.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=b8deddd23f76c286483f3327ad3de7802627e4220eb8890357330f8a13fc0454
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [ ]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 102 ms (started: 2021-04-11 18:48:13 +00:00)


In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 242 ms (started: 2021-04-11 18:48:13 +00:00)


In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 4.45 s (started: 2021-04-11 18:48:14 +00:00)


In [ ]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 27.2 s (started: 2021-04-11 18:48:18 +00:00)


In [ ]:
# Demographic Data Chicago
#!ls drive/MyDrive/big-data-project/data/clean-data/ny/ny.csv

#!ls drive/MyDrive/big-data-project/data/clean-data/ny/social/2019/



time: 4.28 ms (started: 2021-04-11 18:48:45 +00:00)


In [ ]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ny'

time: 1.48 ms (started: 2021-04-11 18:48:45 +00:00)


In [ ]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2019/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+--------------------+------------------------+--------------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_t

In [ ]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [ ]:
# # Take small sample of data to experiment with
# df_soc = df_soc.limit(100)
# df_soc.show()

time: 1.75 ms (started: 2021-04-11 18:49:05 +00:00)


In [ ]:
df_soc.createOrReplaceTempView('clean_ny')
df_soc = spark.sql('SELECT cbg, date_range_start, device_count, completely_home_device_count, part_time_work_behavior_devices, full_time_work_behavior_devices FROM clean_ny')
df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|         cbg|    date_range_start|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|360050414002|2019-09-13T00:00:...|          64|                          19|                              5|                              5|
|360050424001|2019-09-13T00:00:...|          58|                          12|                              4|                              3|
|360470076002|2019-09-13T00:00:...|          68|                          23|                             13|                              6|
|360610164002|2019-09-13T00:00:...|         142|                          42|                             17|                              8|
|36061

In [ ]:
df_soc = spark.sql('SELECT * FROM clean_ny cl WHERE cl.device_count > 50')
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+--------------------+------------------------+--------------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_t

In [ ]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [ ]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+--------------------+------------------------+--------------------------------+--------------------------+-------------------------+-------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior

In [ ]:
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [ ]:
#Create temp view
df_soc.createOrReplaceTempView('new_clean_ny')

time: 20 ms (started: 2021-04-11 18:49:08 +00:00)


In [ ]:
# get mean percentage for each cbg
df_soc_home = spark.sql('SELECT cbg, AVG(completely_home_percentage) AS mean_completely_home_percentage FROM new_clean_ny GROUP BY cbg')
df_soc_home.createOrReplaceTempView('mean_completely_home')
df_soc_home.show()

+------------+-------------------------------+
|         cbg|mean_completely_home_percentage|
+------------+-------------------------------+
|360810152002|              34.41268505324875|
|360050181024|               42.7681834530328|
|360470800001|              37.71870541194492|
|360810278002|              46.42531810119046|
|360810034002|              37.17315610664086|
|360470828001|              40.82393656366318|
|360050413007|             38.334544055606735|
|360850169012|               29.8335483504686|
|360050393006|              46.13814903974811|
|360810838003|             30.428169148821496|
|360810757022|             31.202529010131464|
|360850169011|              28.00142379180564|
|360050157003|              38.38351105479276|
|360810040012|              37.03411483387965|
|360610048004|              27.74953848725689|
|360470349002|              37.82983998814149|
|360610070005|              30.00564947180639|
|360471152001|              43.07753780595931|
|360811367004

TEST FOR JUST COMPLETELY HOME

In [ ]:
df_mobility = df_soc_home

time: 4.39 ms (started: 2021-04-11 18:55:01 +00:00)


In [ ]:
#Get mean percentage for full time work for each cbg
df_soc_ft = spark.sql('SELECT cbg, AVG(full_time_work_percentage) AS mean_full_time_work_percentage FROM new_clean_ny GROUP BY cbg')
df_soc_ft.createOrReplaceTempView('mean_fulltime_work')
df_soc_ft.show()

+------------+------------------------------+
|         cbg|mean_full_time_work_percentage|
+------------+------------------------------+
|360810152002|             6.654530404233535|
|360050181024|             4.252950096315555|
|360470800001|             4.721670357592133|
|360810278002|            3.1403020371012556|
|360810034002|             5.312872207714123|
|360470828001|             3.923666282689755|
|360050413007|             4.782119292331333|
|360850169012|             6.915440305300091|
|360050393006|            3.8544666304548327|
|360810838003|             7.357520340367271|
|360810757022|             9.428201315719383|
|360850169011|             6.878050592129576|
|360050157003|             5.080764517041035|
|360810040012|             5.281603084308337|
|360610048004|              3.88147328488348|
|360470349002|             4.688642431537546|
|360610070005|             5.246881341981688|
|360471152001|             4.584377402134328|
|360811367004|             6.77421

In [ ]:
#Get mean percentage for part time work for each cbg
df_soc_pt = spark.sql('SELECT cbg, AVG(part_time_work_percentage) AS mean_part_time_work_percentage FROM new_clean_ny GROUP BY cbg')
df_soc_pt.createOrReplaceTempView('mean_parttime_work')
df_soc_pt.show()

+------------+------------------------------+
|         cbg|mean_part_time_work_percentage|
+------------+------------------------------+
|360810152002|             8.911254647146913|
|360050181024|             5.003903175030432|
|360470800001|            7.6110256098408735|
|360810278002|             6.240366138539199|
|360810034002|             6.978428236713601|
|360470828001|             6.548817166945385|
|360050413007|              7.49496329119342|
|360850169012|             9.665889498314531|
|360050393006|            5.3473001215822755|
|360810838003|             7.978340763472963|
|360810757022|              8.79691360556808|
|360850169011|            10.574239546479928|
|360050157003|              7.46279871748096|
|360810040012|             7.070809832710609|
|360610048004|            10.152223024174797|
|360470349002|             7.105170913978839|
|360610070005|             8.715545527260439|
|360471152001|             6.924909273107533|
|360811367004|            10.67516

In [ ]:
df_mobility = spark.sql('SELECT mean_completely_home.cbg, mean_completely_home.mean_completely_home_percentage, mean_fulltime_work.mean_full_time_work_percentage , mean_parttime_work.mean_part_time_work_percentage  FROM mean_completely_home INNER JOIN mean_fulltime_work ON mean_completely_home.cbg = mean_fulltime_work.cbg INNER JOIN mean_parttime_work ON mean_completely_home.cbg = mean_parttime_work.cbg')

time: 173 ms (started: 2021-04-11 18:58:15 +00:00)


In [ ]:
#df_mobility.show()
df_mobility.createOrReplaceTempView('mobility')

time: 44.6 ms (started: 2021-04-11 18:58:15 +00:00)


In [ ]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'ny.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|360050001000|        0|               0.0|               0.0|
|360050001001|     7503|               0.0|12.435025989604158|
|360050002000|        0|               0.0|               0.0|
|360050002001|     2114|16.130558183538316|35.856196783349105|
|360050002002|     2168|18.911439114391143| 48.06273062730627|
|360050002003|      969| 8.152734778121776|49.742002063983485|
|360050004000|        0|               0.0|               0.0|
|360050004001|      646|32.972136222910216| 44.27244582043344|
|360050004002|     1400|25.571428571428573| 75.35714285714286|
|360050004003|     3393| 1.569905213270142| 33.65753020925435|
|360050004004|      541|15.196998123827393| 20.33271719038817|
|360050016001|     3427|  25.1531952144733|38.400933761307265|
|360050016002|      662| 62.99093655589124| 73.56495468

In [ ]:
#Join mobility and poverty
result = spark.sql('SELECT demographic.*, mobility.mean_completely_home_percentage, mobility.mean_full_time_work_percentage, mobility.mean_part_time_work_percentage FROM mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
result.createOrReplaceTempView('demographic_mobility')
result.show()

+------------+---------+------------------+------------------+-------------------------------+------------------------------+------------------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|mean_completely_home_percentage|mean_full_time_work_percentage|mean_part_time_work_percentage|
+------------+---------+------------------+------------------+-------------------------------+------------------------------+------------------------------+
|360050076001|      631|27.403846153846157| 9.667194928684628|              46.29199399824631|             4.450219968162994|             6.817880465679207|
|360050157003|     2002|24.925074925074924| 18.03196803196803|              38.38351105479276|             5.080764517041035|              7.46279871748096|
|360050181024|     1457|14.070006863417984| 6.314344543582704|               42.7681834530328|             4.252950096315555|             5.003903175030432|
|360050393006|     1224|  32.6797385620915|2.5326797385620

In [ ]:
# Count number of rows
spark.sql('SELECT COUNT(cbg) FROM demographic_mobility').show()

+----------+
|count(cbg)|
+----------+
|      5817|
+----------+

time: 1min 41s (started: 2021-04-11 19:03:09 +00:00)


In [ ]:
#Sort by poverty
result = spark.sql('SELECT * FROM demographic_mobility ORDER BY CAST(poverty_percentage AS float) DESC')
result.createOrReplaceTempView('demographic_mobility')
result.show()

+------------+---------+------------------+------------------+-------------------------------+------------------------------+------------------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|mean_completely_home_percentage|mean_full_time_work_percentage|mean_part_time_work_percentage|
+------------+---------+------------------+------------------+-------------------------------+------------------------------+------------------------------+
|360610240001|     1742| 89.67653131452168|36.337543053960964|             27.007680629890626|            2.6848964291499984|             7.020649263309882|
|360470910002|     1588| 80.96166341780376| 5.289672544080604|              40.88045801447357|             3.873215092077426|             7.161387488985167|
|360050020003|     3073| 76.88787185354691|42.629352424341036|             43.407665928557776|             3.366845038342742|             6.389265162454425|
|360050399022|      732|              75.0| 5.327868852459

In [ ]:
# group by poverty range


time: 5.09 ms (started: 2021-04-11 19:09:46 +00:00)


In [ ]:
result_df = result.toPandas()
result_df

,cbg,pop_total,poverty_percentage,perc_whiteonly,mean_completely_home_percentage,mean_full_time_work_percentage,mean_part_time_work_percentage
0,360610240001,1742,89.67653131452168,36.337543053960964,27.007681,2.684896,7.020649
1,360470910002,1588,80.96166341780376,5.289672544080604,40.880458,3.873215,7.161387
2,360050020003,3073,76.88787185354691,42.629352424341036,43.407666,3.366845,6.389265
3,360050399022,732,75.0,5.327868852459016,41.908427,4.243071,7.147856
4,360610143001,5,75.0,40.0,50.377935,1.728426,3.722884
...,...,...,...,...,...,...,...
5812,360610066001,1200,0.0,85.16666666666667,31.021348,6.238089,9.560384
5813,360610183007,655,0.0,85.95419847328245,31.716008,4.083978,7.221651
5814,360470890003,468,0.0,0.0,46.019947,3.890090,6.054711
5815,360610086010,0,0.0,0.0,43.185604,1.664754,3.221154


time: 5min (started: 2021-04-11 19:09:46 +00:00)


In [ ]:
# plot bar graph
alt.Chart(result_df).mark_point().encode(
    alt.X('poverty_percentage:Q'),
    alt.Y('mean_completely_home_percentage:Q'),
    tooltip=['cbg','poverty_percentage','mean_completely_home_percentage']
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 289 ms (started: 2021-04-11 19:14:46 +00:00)


In [ ]:
result_mean_df = result_df.groupby(pd.cut(result_df['poverty_percentage'].astype(float), np.arange(0,100, 10))).mean().reset_index()

time: 58.5 ms (started: 2021-04-11 19:14:47 +00:00)


In [ ]:
result_mean_df['poverty_percentage'] = result_mean_df['poverty_percentage'].astype(str)

time: 2.99 ms (started: 2021-04-11 19:14:47 +00:00)


In [ ]:
result_mean_df['color'] = ['#45a0d1' if x>30 else '#ff4833' for x in result_mean_df['mean_completely_home_percentage']]
result_mean_df

,poverty_percentage,mean_completely_home_percentage,mean_full_time_work_percentage,mean_part_time_work_percentage,color
0,"(0, 10]",31.535777,5.965204,8.750521,#45a0d1
1,"(10, 20]",34.719706,5.431103,8.033532,#45a0d1
2,"(20, 30]",36.862557,5.013754,7.622529,#45a0d1
3,"(30, 40]",38.776946,4.437016,7.166626,#45a0d1
4,"(40, 50]",40.137002,4.208654,6.765555,#45a0d1
5,"(50, 60]",40.648816,3.922054,6.446792,#45a0d1
6,"(60, 70]",41.555893,3.884236,6.497161,#45a0d1
7,"(70, 80]",41.999257,3.817775,6.266945,#45a0d1
8,"(80, 90]",33.944069,3.279056,7.091018,#45a0d1


time: 26.2 ms (started: 2021-04-11 19:14:47 +00:00)


In [ ]:
count = result_mean_df.shape[0]
print("{}".format(count))

9
time: 2.23 ms (started: 2021-04-11 19:14:47 +00:00)


In [ ]:
viz_df = []
for col in ['mean_completely_home_percentage','mean_full_time_work_percentage','mean_part_time_work_percentage']:
        for c in range(0,count):
            viz_df.append({'poverty': c, 'column': col, 'value': result_mean_df[col][c]})
viz_df = alt.Data(values=viz_df)

time: 7.69 ms (started: 2021-04-11 19:14:47 +00:00)


In [ ]:
alt.Chart(result_mean_df).mark_bar(size=30).encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_completely_home_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_completely_home_percentage')]
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 32 ms (started: 2021-04-11 19:14:47 +00:00)


In [ ]:
#
alt.Chart(result_mean_df).mark_point().encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_completely_home_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_completely_home_percentage')]
).properties(width=400, height=400).interactive()



alt.Chart(...)

time: 29.9 ms (started: 2021-04-11 19:14:47 +00:00)


In [ ]:
result_df['poverty_percentage'].astype(float).dtype

dtype('float64')

time: 7.39 ms (started: 2021-04-11 19:14:47 +00:00)


In [ ]:
#
alt.Chart(result_mean_df).mark_point().encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_full_time_work_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_full_time_work_percentage')]
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 22.6 ms (started: 2021-04-11 19:24:45 +00:00)


In [ ]:
#
alt.Chart(result_mean_df).mark_point().encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_part_time_work_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_part_time_work_percentage')]
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 21.2 ms (started: 2021-04-11 19:25:33 +00:00)
